In [15]:
import pyedflib
import numpy as np
%matplotlib inline

# Reading the EDF file containing EEG Record

p1 = pyedflib.EdfReader(r'''C:\Users\lenovo\Desktop\altera\SVM\CHB\Patient1\chb01_21.edf''')
n = p1.signals_in_file
signal_labels = p1.getSignalLabels()
sigbufs = np.zeros((n, p1.getNSamples()[0]))
for i in np.arange(n):
    sigbufs[i, :] = p1.readSignal(i)
print(n)

23


In [18]:
import pandas as pd
import scipy.stats
from scipy.signal import butter, lfilter,freqz, firwin
import numpy as np

# Extracting feature from the EEG records read

fs = 256 #Sampling Frequency
fc = np.linspace(0.5,25,3) #Frequecy Range
n = 1001 #Filter order
x = input('How many feature vectors?\n')
s = input('Start from which second?\n')
ch = p1.signals_in_file #Number of channels
label = input('Label of feature vector?\n') #Label for seizure and non-seizure feature vector extracted
Xt = np.empty((0,ch*8+1), int)
s = int(s)

# Creating the matrix used for training dataset

for k in range(0,int(x)):
    X2 = np.array([])
    X1 = np.array([])
    for channel in range(0,int(ch)):
        sample = s*fs
        y1 = sigbufs[channel]
        y1 = y1[sample:sample+256]
        X1 = np.array([np.mean(y1),np.median(y1),np.array(scipy.stats.mode(y1))[0],np.std(y1),np.amin(y1),np.amax(y1)])
        for i in range(0,2):
            a = firwin(n,[fc[i], fc[i+1]], window = 'nuttall',pass_zero = False,nyq = fs) #Used FIR Filter
            y_filter = lfilter(a,1,y1)
            e = sum(np.absolute(y_filter)**2)
            X1 = np.append(X1,e)
        X2 = np.append(X2,X1)
    X2 = np.append(X2,int(label))
    Xt = np.append(Xt,[X2],axis = 0)
    s = s+1
    
    
# Transferring the data to a test file
# Will be copied to Original CSV dataset 'PythonData.CSV' when structing training data

file = 'C:\\Users\\lenovo\\Desktop\\test.xlsx'    
a = pd.DataFrame(Xt)
a.to_excel(file,index=False, index_label=None,sheet_name='1', header = None)

How many feature vectors?
10
Start from which second?
343
Label of feature vector?
0


In [19]:
from sklearn import svm
import csv
import numpy as np
from sklearn.preprocessing import *
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
import pandas as pd

# Designing the SVM classifier and detecting seizure

filename = 'C:\\Users\\lenovo\\Desktop\\Study\\Master Thesis\\CHB\\Patient1\\PythonData.csv'     # Support Vectors
data = pd.read_csv(filename, header  = None)
X = data.iloc[:,0:ch*8].values
y = data.iloc[:,ch*8].values
scaler = StandardScaler().fit(X)
X = scaler.transform(X)  #Scaling the Training Dataset
filename2 = 'C:\\Users\\lenovo\\Desktop\\test.xlsx'    # Test Vectors
data2 = pd.read_excel(filename2, header =None)
X2 = data2.iloc[:,0:ch*8].values
X2 = scaler.transform(X2)  #Scaling the Test Datasets

# Designing the classifier by defining kernel parameters and fitting with Training Dataset
clf = svm.SVC(kernel = 'rbf', gamma = .001,C = 100).fit(X,y)

#Printing the prediction data
print('Detection Result:\n',clf.predict(X2))

#Writing the prediction result in excel file to verify false alarm      
file = 'C:\\Users\\lenovo\\Desktop\\FA.xlsx'    
b = pd.DataFrame(clf.predict(X2))
b.to_excel(file,index=False, index_label=None,sheet_name='1', header = None)


#Finding the parameters of SVM classifier
print('Bias:',clf.intercept_)                #Value of bias(b)
iteration = clf.dual_coef_.shape[1:2]
print('Number of Iteration:',iteration)      #Value of i
print('Values of alpha:\n',clf.dual_coef_)   #Value of alpha

#Checking stats on support vectors
s = clf.n_support_
print('No of Support Vectors used for seizure:',s[1])
print('No of Support Vectors used for non seizure:',s[0])
print('Array of Support Vectors:\n',clf.support_vectors_)           #The value of x
print('Labels of the Support Vectors:\n',y[(clf.support_)])         #The vaue of y

Detection Result:
 [1 1 1 1 1 1 1 1 1 1]
Bias: [-2.30191565]
Number of Iteration: (233,)
Values of alpha:
 [[ -1.17909355e+01  -2.24084081e+01  -1.60685727e+01  -4.15566889e+00
   -4.49533817e+00  -6.87378791e-01  -1.07818657e+01  -4.71256822e+01
   -3.00344395e+01  -4.31734381e+01  -4.12486170e+00  -2.16504200e+01
   -3.65531555e+01  -1.08557648e+01  -7.68199404e+00  -5.33108893e+00
   -1.08408522e+01  -4.97285515e+00  -4.19703483e+00  -1.74617768e+01
   -2.89916117e+01  -1.28504403e+01  -4.40186594e+00  -1.52821924e+01
   -1.08518522e+01  -2.48324628e+01  -1.57690840e+01  -8.69913365e-01
   -9.92571405e+00  -2.08244259e+01  -1.18996545e+00  -1.08130957e+01
   -1.15877685e+00  -1.80868465e+01  -6.88397086e+00  -9.57539707e+00
   -4.26969563e+01  -1.89708954e+01  -1.51213503e+01  -4.08929276e+01
   -1.94026849e+01  -5.57426919e-01  -1.14811401e+01  -1.54067183e+01
   -1.50866501e+01  -7.64049014e+00  -8.91261652e+00  -2.00640694e+01
   -2.09061947e+01  -1.92494703e+00  -1.72083276e+01 

In [ ]:
#Using Grid Search method to select kernel and parameters

from sklearn import svm
import csv
import numpy as np
from sklearn.preprocessing import *
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
import pandas as pd
filename = 'C:\\Users\\lenovo\\Desktop\\CHB\\Patient1\\PythonData.csv'
data = pd.read_csv(filename, header = None)
X = data.iloc[:,0:ch*8].values
y = data.iloc[:,ch*8].values
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [.1, .05, 1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 50, 100, 1000]}]

clf = GridSearchCV(SVC(),tuned_parameters,cv = 5, scoring = 'accuracy').fit(X,y)
print(clf.best_score_)
print(clf.best_params_)
print(clf.best_estimator_)
z = pd.DataFrame(clf.grid_scores_)[['parameters','mean_validation_score']]
print(z)

In [21]:
#Saving the training and testing dataset in excel file

file = 'C:\\Users\\lenovo\\Desktop\\Study\\Master Thesis\\CHB\\Python\\MasterSV1.xlsx' 
file2 = 'C:\\Users\\lenovo\\Desktop\\Study\\Master Thesis\\CHB\\Python\\MasterAlpha1.xlsx'   
file3 = 'C:\\Users\\lenovo\\Desktop\\Study\\Master Thesis\\CHB\\Python\\MasterTest1.xlsx'
a = pd.DataFrame(clf.support_vectors_)
a.to_excel(file,index=False, index_label=None,sheet_name='sv', header = None)
b = pd.DataFrame(clf.dual_coef_)
b.to_excel(file2,index=False, index_label=None,sheet_name='alpha', header = None)
c = pd.DataFrame(X2)
c.to_excel(file3,index=False, index_label=None,sheet_name='z', header = None)